In [2]:
import pandas as pd
import numpy as np

In [5]:
employee = pd.read_csv('../diversity/employee.csv')

In [6]:
employee.head()

,employee_id,signing_bonus,salary,degree_level,sex,yrs_experience
0,138719,0,273000.0,Master,M,2
1,3192,0,301000.0,Bachelor,F,1
2,114657,0,261000.0,Master,F,2
3,29039,0,86000.0,High_School,F,4
4,118607,0,126000.0,Bachelor,F,3


In [15]:
employee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   employee_id     10000 non-null  int64  
 1   signing_bonus   10000 non-null  int64  
 2   salary          10000 non-null  float64
 3   degree_level    10000 non-null  object 
 4   sex             10000 non-null  object 
 5   yrs_experience  10000 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 468.9+ KB


In [24]:
employee[['degree_level', 'salary']].groupby('degree_level').mean().round()

,salary
degree_level,
Bachelor,187939.0
High_School,188089.0
Master,188145.0
PhD,191804.0


In [8]:
company = pd.read_csv('../diversity/company_hierarchy.csv')

In [9]:
company.head()

,employee_id,boss_id,dept
0,46456,175361.0,sales
1,104708,29733.0,HR
2,120853,41991.0,sales
3,142630,171266.0,HR
4,72711,198240.0,sales


In [12]:
company['dept']


array(['sales', 'HR', 'engineering', 'marketing', 'CEO'], dtype=object)

In [26]:
merged = pd.merge(employee, company, how='inner', on=['employee_id'])

In [27]:
merged.head()

,employee_id,signing_bonus,salary,degree_level,sex,yrs_experience,boss_id,dept
0,138719,0,273000.0,Master,M,2,43602.0,engineering
1,3192,0,301000.0,Bachelor,F,1,87847.0,sales
2,114657,0,261000.0,Master,F,2,180854.0,sales
3,29039,0,86000.0,High_School,F,4,88370.0,HR
4,118607,0,126000.0,Bachelor,F,3,23565.0,sales


In [ ]:
#I could using signing bonus?